<a href="https://colab.research.google.com/github/rajmohanAI/iambuddy/blob/main/IAMBuddy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install llama_index
!pip install streamlit -q
!pip install llama-index-llms-huggingface
!pip install llama-index-llms-huggingface-api
!pip install pypdf
!pip  install docx2txt
!pip install langchain_community
!pip install llama_index.embeddings.huggingface

!pip install llama_index.llms.groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.4/253.4 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.7/300.7 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 49.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 54.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 78.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 97.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 81.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 

In [7]:
%%writefile app.py

from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, Settings
import streamlit as st
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.vector_stores import MetadataFilters, ExactMatchFilter
import os
import uuid


# Settings
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
global_query_engine = None

# LLM
from llama_index.llms.groq import Groq
Settings.llm = Groq(model="mixtral-8x7b-32768", api_key="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")


# Load files and get as documents
def load_files(files):
  UPLOAD_DIR = str(uuid.uuid4())
  os.makedirs(UPLOAD_DIR, exist_ok=True)

  print(f"GUID is {UPLOAD_DIR}")

  for uploaded_file in files:
        file_path = os.path.join(UPLOAD_DIR, uploaded_file.name)
        with open(file_path, "wb") as f:
            f.write(uploaded_file.getbuffer())


  # todo
  documents = SimpleDirectoryReader(UPLOAD_DIR).load_data()
  return documents

def add_documents_to_index(index, documents, channel_id):
  pipeline = IngestionPipeline(
      transformations=[
          SentenceSplitter(chunk_size=512, chunk_overlap=20),
          ]
      )

  for document in documents:
    document.metadata["channel_id"] = channel_id

  nodes = pipeline.run(documents=documents)
  index.insert_nodes(nodes)
  return index

def get_query_engine(index, channel_id):
  query_engine = index.as_query_engine(
      filters = MetadataFilters(
          filters=[
              ExactMatchFilter(
                  key = "channel_id",
                  value = channel_id,
                  )
              ]
          ),
      similarity_top_k=3,
      )
  return query_engine

def user_prompt(index, channel_id, prompt_query):
  query_engine = get_query_engine(index, channel_id)
  reply = query_engine.query(prompt_query)
  return f"Question:\n{prompt_query}\n\nReply is:\n{reply.response}"

def open_streamlit():
  st.set_page_config("Chat PDF")
  st.header("Chat with PDF")

  # Dropdown for selecting Channel ID instead of text input
  st.session_state.channel_id = st.selectbox(
      "Select an IAM Suite ",
      ["Forgerock", "Sailpoint", "Okta"]
  )

  # Initialize index in session state if not already set
  if "index" not in st.session_state:
      st.session_state.index = VectorStoreIndex.from_documents([], embed_model=embed_model)

  user_question = st.text_input("Ask a Question from the PDF Files")
  if user_question:
    st.write("Searching...")
    reply = user_prompt(st.session_state.index, st.session_state.channel_id, user_question)
    st.write(reply)
    st.session_state.user_question = ""

  with st.sidebar:
        st.title("Menu:")
        user_docs = st.file_uploader("Upload your PDF Files and Click on the Submit & Process Button", accept_multiple_files=True)
        if st.button("Submit & Process"):
            with st.spinner("Processing..."):
                documents = load_files(user_docs)
                st.session_state.index = add_documents_to_index(st.session_state.index, documents, st.session_state.channel_id)
                st.success("Done")

open_streamlit()

Overwriting app.py


In [8]:
# getting the ip to run streamlit
!wget -q -O - ipv4.icanhazip.com

34.139.95.167


In [ ]:
# run this command to open streamlit
!streamlit run app.py & npx localtunnel --port 8501



⠙⠹
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.139.95.167:8501

⠸⠼⠴your url is: https://wise-towns-yell.loca.lt
2025-02-22 14:07:21.161248: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740233241.186711    7763 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740233241.194799    7763 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-22 14:07:21.221299: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following 